In [212]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import cv2
import time                                                                 #Import all libraries
# from gtts import gTTS
import os

In [213]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml') # Object of face detector

roi_gray = []

#Removes parts of the sides of the face
#This is done so that the algorithm has to work with only the relevant/ most important part of the image
def cut_faces(image, faces_coord):                                          
    faces = []

    for (x, y, w, h) in faces_coord:                                        #Trims parts of the face
        w_rm = int(0.2 * w / 2)
        faces.append(image[y : y + h, x + w_rm :  x + w - w_rm])
        
    return faces                                                            #Returns co-ordinates of the face

In [214]:
#Adds a new person to the dataset and creates a separate folder for them
    

    
         #Get the name of the new person
def add_person2(person_name):

    
    folder = 'people_folder' +'/'+ person_name                              
    
    if not os.path.exists(folder):
        #f3=Frame(bg="#091e42")
        #f3.place(x=0,y=0,width=700,height=700)
    
        root.destroy()
        #Find the if the data for the given person already exists
        #l8=Label(f3,text="i am taking 50 pictures",font=("arial",25))
        #l8.grid(row=0,column=0,pady=25)
       

        os.makedirs(folder)                                                    # Makes the new folder for saving the photos
        
        video = cv2.VideoCapture(0)
        detector = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml') #Loads the HAAR cascade to detect faces

        counter = 1
        timer = 0
        
        cv2.namedWindow('Video Feed', cv2.WINDOW_AUTOSIZE)
        cv2.namedWindow('Saved Face', cv2.WINDOW_NORMAL)                    
        
        while counter < 51:
            _, frame = video.read()

            
            if counter == 1:
                time.sleep(6)
            else:
                time.sleep(1)
                
            faces = detector.detectMultiScale(frame)                        #Finding the co-ordinates of all faces in the frame
            
            if len(faces):                                                  #If we have some faces

        
                cut_face = cut_faces(frame, faces)                          #Remove the unecessary part of the face
                
                face_bw = cv2.cvtColor(cut_face[0], cv2.COLOR_BGR2GRAY)
                
                face_bw_eq = cv2.equalizeHist(face_bw)                      # Histogram equalization 
                face_bw_eq = cv2.resize(face_bw_eq, (256, 256), interpolation = cv2.INTER_CUBIC)#Resizing the image to 100 x 100 pixels
                #cv2.imshow('Face Recogniser', face_bw_eq)


                cv2.imwrite(folder + '/' + str(counter) + '.png',
                            face_bw_eq)
                print('Images Saved:' + str(counter))
                counter += 1
                cv2.imshow('Saved Face', face_bw_eq)                        #Display the face that has been saved

            cv2.imshow('Video Feed', frame)
            cv2.waitKey(50)

    else:
        print("This name already exists.")                                  # If the person already exists


In [215]:
#Does the face recognition in real time
#Pressing ESC closes the live recognition
def live():
    
    cv2.namedWindow('Predicting for')
    images = []
    labels = []
    labels_dic = {}
    people = [person for person in os.listdir("people_folder")]
    threshold = 37                                                  #Threshold for the Face recognizer algorithm/ permissible distance
                                                                            #from another face


    for i, person in enumerate(people):
        print(person)
        labels_dic[i] = person
        
        for image in os.listdir("people_folder/" + person):
            print(image)
            images.append(cv2.imread('people_folder/'+person+'/'+ image, 0))
            labels.append(i)
            
    labels = np.array(labels)
    
    #rec_eig = cv2.face.EigenFaceRecognizer_create()
    rec_lbhp = cv2.face.LBPHFaceRecognizer_create()                         #Creates a LBHP face recognizer object
    
    rec_lbhp.train(images, labels)                                          #Trains the model
    
    cv2.namedWindow('face')
    webcam = cv2.VideoCapture(0)
    while True:
        _, frame = webcam.read()
        
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)                #Gets the co-ordinates of the face in the frame
        
        if len(faces):
            cut_face = cut_faces(frame, faces)                              #Trims the face to feed it to our predictive model
            
            face = cv2.cvtColor(cut_face[0], cv2.COLOR_BGR2GRAY)
            face = cv2.equalizeHist(face)                                   #Histogram Equalization
            face = cv2.resize(face, (256, 256), interpolation = cv2.INTER_CUBIC)#Resizes the image of the face
            
            cv2.imshow('face', face)
                              
            collector = cv2.face.StandardCollector_create()
            rec_lbhp.predict_collect(face, collector)
            conf = collector.getMinDist()                                   #Finds the face with the closest proximity to our given face
                              
            print('Confidence ', conf)
            pred = collector.getMinLabel()
            txt = ''
            
            if conf < threshold:                                            #If a matching face is found
                txt = labels_dic[pred].upper()                              #Get the name of the person                 
            else:
                txt = 'Uknown'                                              #If unrecognised, label as Unknown
                              
            cv2.putText(frame, txt,
                        (faces[0][0], faces[0][1] - 10),
                        cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)        #Puts the text on the current frame
                              
            print(faces)
            cv2.rectangle(frame, (faces[0][0], faces[0][1]),(faces[0][0] + faces[0][2], faces[0][1] + faces[0][3]), (255, 255, 0), 8)#Makes rectangle around face
                              
            cv2.putText(frame,"ESC to exit", (5, frame.shape[0] - 10),
                        cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2, cv2.LINE_AA)

        cv2.imshow("Live", frame)                                           #Displays the frame

        if cv2.waitKey(20) & 0xFF == 27:
            cv2.destroyAllWindows()
            break


In [216]:
from tkinter import *
from tkinter import simpledialog
root=Tk()
root.geometry("700x700")
root.resizable(0,0)




def show():
    s1=e1.get()
    s2=e2.get()
    print(s1)
    if (s1=="face" and s2=="f"):
        f2=Frame(root,bg="#091e42")
        f2.place(x=0,y=0,width=700,height=700)
        l3=Label(f2,text="welcome",font=("arial",25))
        l3.grid(row=0,column=0,pady=25,sticky=W)
        b2=Button(f2,text="add a face",font=("arial",25),command=add_person)
        b2.place(x=350,y=120,anchor=CENTER)
        b3=Button(f2,text="delete face",font=("arial",25))
        b3.place(x=350,y=200,anchor=CENTER)
        b4=Button(f2,text="settings",font=("arial",25))
        b4.place(x=350,y=280,anchor=CENTER)
        b5=Button(f2,text="go live",font=("arial",25),command=live)
        b5.place(x=350,y=360,anchor=CENTER)
        b6=Button(f2,text="exit",font=("arial",25))
        b6.place(x=350,y=440,anchor=CENTER)
       
        
        
        
        
        


    
        
    else:
        l4=Label(f1,text="incorrect password",font=("arial",22))
        l4.grid(row=4,column=0,pady=25,sticky=W)
        b7=Button(f1,text="retry",font=("arial",25))
        b7.grid(row=5,column=0,pady=25,columnspan=2,)
        
def add_person():
    #l9=Label(f3,text="what is the  name of the person",font=("arial",25))
    #l9.grid(row=0,column=0,pady=25,sticky=W)
    nameentry=simpledialog.askstring(title="window",prompt="name person")
    #nameentry.grid(row=3,column=0,pady=25)
    #person=nameentry.get()
    print (nameentry)
    add_person2(nameentry)
    
    #b6=Button(f3,text="enter",font=("arial",25),command=lambda: add_person2(nameentry))
    #b6.grid(row=4,column=0,pady=25)
    



f1=Frame(root,bg="#091e42")
f1.place(x=0,y=0,width=700,height=700)

l1=Label(f1,text="enter name",font=("arial",25))
l1.grid(row=0,column=0,pady=25,sticky=W)
e1=Entry(f1,font=("arial",25))
e1.grid(row=0,column=1,pady=25)


l2=Label(f1,text="enter Password",font=("arial",25))
l2.grid(row=1,column=0,pady=25)
e2=Entry(f1,show='*',font=("arial",25))
e2.grid(row=1,column=1,pady=25)


b1=Button(f1,text="login",font=("arial",25),command=show)
b1.grid(row=2,column=0,pady=25,columnspan=2)



    #b0=Button(root,text="click",font=("arial",25),command=home)
    #b0.grid(row=2,column=0,pady=25)    

    #house()

root.mainloop()
    
    



cv2.destroyAllWindows()

face
vishvesh
Images Saved:1
Images Saved:2
Images Saved:3
Images Saved:4
Images Saved:5
Images Saved:6
Images Saved:7
Images Saved:8
Images Saved:9
Images Saved:10
Images Saved:11
Images Saved:12
Images Saved:13
Images Saved:14
Images Saved:15
Images Saved:16
Images Saved:17
Images Saved:18
Images Saved:19
Images Saved:20
Images Saved:21
Images Saved:22
Images Saved:23
Images Saved:24
Images Saved:25
Images Saved:26
Images Saved:27
Images Saved:28
Images Saved:29
Images Saved:30
Images Saved:31
Images Saved:32
Images Saved:33
Images Saved:34
Images Saved:35
Images Saved:36
Images Saved:37
Images Saved:38
Images Saved:39
Images Saved:40
Images Saved:41
Images Saved:42
Images Saved:43
Images Saved:44
Images Saved:45
Images Saved:46
Images Saved:47
Images Saved:48
Images Saved:49
Images Saved:50
